In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
import pandas as pd
import seaborn as sns
from IPython.display import display
from scipy.stats import gaussian_kde

from sklearn.compose import ColumnTransformer , make_column_transformer, TransformedTargetRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.model_selection import cross_val_score, KFold, StratifiedShuffleSplit, GridSearchCV
from sklearn.pipeline import Pipeline,make_pipeline

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer,PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,QuantileTransformer,PowerTransformer
from sklearn.datasets import make_classification
from sklearn.metrics import precision_score, classification_report,log_loss
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,mean_absolute_percentage_error,f1_score,accuracy_score, r2_score, mean_squared_log_error, mean_squared_error, mean_absolute_error
import category_encoders as ce
#pip install category-encoders
from category_encoders import TargetEncoder
from sklearn.linear_model import ElasticNet, Ridge, Lasso, Lars,SGDRegressor,LogisticRegression
from sklearn.svm import SVR,SVC

from sklearn.ensemble import VotingRegressor,VotingClassifier,RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,StackingClassifier,GradientBoostingRegressor,RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier


from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn.datasets import load_wine
from sklearn import svm
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler,LabelEncoder,FunctionTransformer



In [4]:
path_train = 'train_house_B.csv'

In [5]:
df = pd.read_csv(path_train)
display(df.shape)

(408, 30)

In [6]:
cor =df.corr()

In [20]:
feat = cor[df.corr()>0.3]['price'].dropna().index
feat

Index(['price', '2', '12', '18', '19', '20', '21', '22', '24', '25', '27'], dtype='object')

In [21]:
X = df.drop(columns=['price'])
y = (df.price)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42 )

In [36]:
selected_feat = ['2', '12', '18', '19', '20', '21', '22', '24', '25', '27']

X_trainsel = X_train[selected_feat]
X_testsel = X_test[selected_feat]


In [37]:
coltrans = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), selected_feat)],

    remainder='drop'
).set_output(transform="pandas")

display(coltrans)

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['2', '12', '18', '19', '20', '21', '22', '24',
                                  '25', '27'])])

In [38]:
model_1 = Pipeline(steps = [
                          ('preproc',coltrans),
                        ('estimator',RandomForestRegressor(n_estimators =500,
                                                              min_samples_split=5,
                                                               n_jobs = -1,
                                                            max_depth = 2,
                                                               random_state=42))
                        ])


model_1.fit(X_trainsel,y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['2', '12', '18', '19', '20',
                                                   '21', '22', '24', '25',
                                                   '27'])])),
                ('estimator',
                 RandomForestRegressor(max_depth=2, min_samples_split=5,
                                       n_estimators=500, n_jobs=-1,
                                       random_state=42))])

In [39]:
y_pred_1 = model_1.predict(X_test)
mape_test_1 = mean_absolute_percentage_error(y_test, y_pred_1)
print(f'MAPE_TEST: {mape_test_1}')

MAPE_TEST: 0.24193695287076805


In [40]:
data = coltrans.fit_transform(X_train)
data

,num__2,num__12,num__18,num__19,num__20,num__21,num__22,num__24,num__25,num__27
158,-1.398862,-1.453432,-0.240814,-0.148213,-0.554382,0.052408,1.378979,-0.761059,0.165592,-0.707107
109,0.675727,0.808443,-0.240814,1.416258,1.803810,0.052408,1.378979,-0.761059,1.316659,1.414214
17,-1.143778,-0.204715,-0.240814,-0.148213,-0.554382,-1.305443,-0.538138,0.374553,-0.985475,1.414214
237,-0.801808,-0.204715,-0.240814,-0.148213,-0.554382,-1.305443,-0.538138,0.374553,0.165592,-0.707107
24,-0.532320,-0.508662,1.777774,-0.148213,-0.554382,1.410260,1.378979,-0.761059,0.165592,-0.707107
...,...,...,...,...,...,...,...,...,...,...
71,0.071704,0.149891,1.777774,-1.712684,1.803810,1.410260,1.378979,-0.761059,-0.985475,1.414214
106,0.443410,0.555154,-0.240814,-1.712684,1.803810,0.052408,1.378979,1.510164,-0.985475,-0.707107
270,-0.063040,-0.204715,-0.240814,-0.148213,-0.554382,0.052408,-0.538138,-0.761059,0.165592,-0.707107
348,0.968911,-0.204715,-0.240814,-0.148213,1.803810,0.052408,-0.538138,-0.761059,0.165592,1.414214


In [41]:
rf = RandomForestRegressor(n_estimators =500,min_samples_split=5,n_jobs = -1,max_depth = 5,
                                                               random_state=42)
rf.fit(data,y_train)

RandomForestRegressor(max_depth=5, min_samples_split=5, n_estimators=500,
                      n_jobs=-1, random_state=42)

In [42]:
n_feat = data.shape[1]
ind = np.argsort(rf.feature_importances_)[::-1]
sorted_i = rf.feature_importances_[ind]
sorted_f = data.columns[ind]
sorted_f

Index(['num__12', 'num__2', 'num__25', 'num__24', 'num__20', 'num__22',
       'num__21', 'num__27', 'num__18', 'num__19'],
      dtype='object')

In [43]:
selected_f2 = [ '12', '2', '25', '24', '20', '22' , '21', '27','18','19']
X_trainsel2 = X_train[selected_f2]
X_testsel2 = X_test[selected_f2]

In [44]:
coltrans_2 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), selected_f2)],

    remainder='drop'
).set_output(transform="pandas")

display(coltrans)

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['2', '12', '18', '19', '20', '21', '22', '24',
                                  '25', '27'])])

In [45]:
model_baseline_3 = TransformedTargetRegressor(
                        regressor = Pipeline(steps = [
                          ('preproc',coltrans_2),
                        ('estimator',SVR(kernel = 'linear'))
                        ]),
    transformer =PowerTransformer() )


model_baseline_3.fit(X_trainsel2,y_train)

TransformedTargetRegressor(regressor=Pipeline(steps=[('preproc',
                                                      ColumnTransformer(transformers=[('num',
                                                                                       StandardScaler(),
                                                                                       ['12',
                                                                                        '2',
                                                                                        '25',
                                                                                        '24',
                                                                                        '20',
                                                                                        '22',
                                                                                        '21',
                                                                                        '27',
                                                                                        '18',
                                                                                        '19'])])),
                                                     ('estimator',
                                                      SVR(kernel='linear'))]),
                           transformer=PowerTransformer())

In [46]:
y_pred = model_baseline_3.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE_TEST: {mape}')

y_pred = model_baseline_3.predict(X_train)
mape = mean_absolute_percentage_error(y_train, y_pred)
print(f'MAPE_TRAIN: {mape}')

MAPE_TEST: 0.1920230655112644
MAPE_TRAIN: 0.1805904067211769
